<a href="https://colab.research.google.com/github/MatteoRigoni/MachineLearningPlayground/blob/master/Progetto_CrossSelling_MatteoRigoni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Contesto

> Problema di classificazione binaria

Il cliente è una compagnia di assicurazioni che ha fornito un'assicurazione sanitaria ai suoi clienti, adesso hanno bisogno del tuo aiuto per costruire un modello predittivo in grado di prevedere se gli assicurati dell'anno passato potrebbero essere interessati ad acquistare anche un'assicurazione per il proprio veicolo.

Il dataset è composto dalle seguenti proprietà:

* id: id univoco dell'acquirente.
* Gender: sesso dell'acquirente.
* Age: età dell'acquirente.
* Driving_License: 1 se l'utente ha la patente di guida, 0 altrimenti.
* Region_Code: codice univoco della regione dell'acquirente.
* Previously_Insured: 1 se l'utente ha già un veicolo assicurato, 0 altrimenti.
* Vehicle_Age: età del veicolo
* Vehicle_Damage: 1 se l'utente ha danneggiato il veicolo in passato, 0 altrimenti.
* Annual_Premium: la cifra che l'utente deve pagare come premio durante l'anno.
* Policy_Sales_Channel: codice anonimizzato del canale utilizzato per la proposta (es. per email, per telefono, di persona, ecc...)
* Vintage: numero di giorni dalla quale l'utente è cliente dell'azienda.
* Response: 1 se l'acquirente ha risposto positivamente alla proposta di vendita, 0 altrimenti.


> L'obiettivo del modello è prevedere il valore di Response.



---



### Lettura del dataset

Si legge il dataset, leggendo la prima colonna come indice e si valutano le classi sbilanciate tramite il comando *value_counts*

Emerge che la classe "Response" è molto sbilanciata, quindi sono molti di più coloro che non hanno aderito alla proposta di vendita

Si imposta un random_seed per i successivi calcoli in modo da avere sempre gli stessi valori

In [ ]:
import pandas as pd

RANDOM_SEED = 2

df = pd.read_csv('https://raw.githubusercontent.com/MatteoRigoni/MachineLearningPlayground/master/insurance_cross_sell.csv', index_col=0)

print(df.head())

print(df['Response'].value_counts())

### Preprocessing del dataset

Di seguito si trasformano le variabili categoriche in numeriche.
Alcune si mappano manualmente in modo da dare un significato preciso alle categorie.

Tramite *df.count()* si evince che non ci sono dati mancanti da compensare

In seguito si effettua la separazione delle features dalla variabile target, droppando anche la colonna dell'id che non è significativa all'analisi

Infine si genera un train_set ed un test_set, effettuando la standardizzazione dei dati

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Vehicle_Age'] = df['Vehicle_Age'].map({ '< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2})
df['Vehicle_Damage'] = df['Vehicle_Damage'].map({ 'Yes': 1, 'No': 0})

print(df.count())

x = df.drop(['Response'], axis=1)
y = df['Response']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_SEED)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

## Definizione di vari modelli da verificare

Si generano diversi modelli, con regressione lineare, regressione polinomiale applicando diverse tecniche di bilanciamento.

Si provano le seguenti opzioni:

* nessuna tecnica di bilanciamento
* SMOTE: dà maggiore importanza alla classe minotaria
* Undersampling: riduce numero di esempi per la classe prevalente

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

poly = PolynomialFeatures(degree=2, include_bias=False)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.fit_transform(x_test)

models = {
    "Reg. lineare": LogisticRegression(random_state=RANDOM_SEED),
    "Reg. lineare (balanced)": LogisticRegression(random_state=RANDOM_SEED, class_weight='balanced')
}

sampling_patterns = {
    "No sampling": None,
    "SMOTE": SMOTE(random_state=RANDOM_SEED),
    "UnderSampling": RandomUnderSampler(random_state=RANDOM_SEED)
}

## Definizione di un metodo per la valutazione delle caratteristiche del modello

Si valutano:
* roc_curve: per avere valutazione visiva del modello
* auc: per stimare capacità del modello di predire il risultato (da 0 a 1, valore maggiore è meglio)
* confusion_matrix: per vedere quanti falsi negativi/positivi ci sono (e viceversa)
* class_report: PRECISION, RECALL, F1, ACCURACY
* cross-validation: per verificare il comportamento al variare del training/test set, su un numero di 5 suddivisioni


In [ ]:
from sklearn.model_selection import cross_val_score

def evaluate_model(model, x_train, y_train, x_test, y_test, name_model, threshold=0.5):
  y_pred_proba = model.predict_proba(x_test)[:, 1]
  y_pred = (y_pred_proba >= threshold).astype(int)

  conf_matrix = confusion_matrix(y_test, y_pred)
  class_report = classification_report(y_test, y_pred)

  fp, tp, _ = roc_curve(y_test, y_pred_proba)
  auc_score = auc(fp, tp)

  cv_scores = cross_val_score(model, x_train, y_train, cv=5, scoring='roc_auc')

  return {
      'fp': fp,
      'tp': tp,
      'AUC': auc_score,
      'CrossValidationAUC': cv_scores.mean(),
      'ConfusionMatrix': conf_matrix,
      'ClassificationReport': class_report
  }

##Addestramento e valutazione dei modelli
Si combinano diverse tecniche per la gestione del dataset sbilanciato.

In [ ]:
results = []

for name_model, model in models.items():
  for sampling_name, sampling in  sampling_patterns.items():
    if sampling:
      x_train_balanced, y_train_balanced = sampling.fit_resample(x_train, y_train)
    else:
      x_train_balanced, y_train_balanced = x_train, y_train

    model.fit(x_train_balanced, y_train_balanced)

    result_evaluation_model = evaluate_model(model, x_train_balanced, y_train_balanced, x_test, y_test, f"{name_model} with {sampling_name}")

    results.append({
        "Model": name_model,
        "Sampling": sampling_name,
        **result_evaluation_model
    })

## Visualizzazione dei risultati

Si confrontano visivamente e tramite valori i risultati delle combinazioni analizzate tenendo conto delle seguenti metriche:

* AUC: area sotto la curva ROC, in generale un valore più vicino a 1 indica una migliore previsione da parte del modello. Sotto lo 0.5 non è significativo
* Cross-validation AUC: media delle AUC su diversi traning/test set, in modo da essere certi che la buona predizione dell'AUC non sia dia da un divisione "fortunata" del dataset
* Confusion matrix: numero di veri positivi (TP), fasi negativi (FP), veri negativi (TN) e falsi negativi (FN):

    [TN  FP]

    [FN  TP]

* Classification report:
  * Precision: % di predizioni positive corrette (più alto è meglio)
  * Recall: % di effettivi positivi predetti dal modello (più alto è meglio)
  * F1: Parametro di valutazione che si basa sia si precision che recall (più alto è meglio)
  * Support: Numero di istanze per classe
  * Accuracy: % di predizioni corrette (più alto è meglio)
  * Macro average / Wighted Average: Media aritmetica/ponderata delle metriche delle due classi (più alto è meglio)

In [ ]:
import matplotlib.pyplot as plt

for result in results:
    print(f"Model: {result['Model']} with sampling: {result['Sampling']}")
    print(f"AUC: {result['AUC']}")
    print(f"Cross-Validation AUC: {result['CrossValidationAUC']}")
    print(f"Confusion Matrix:\n{result['ConfusionMatrix']}")
    print(f"Classification Report:\n{result['ClassificationReport']}")

    plt.figure()
    plt.plot(result['fp'], result['tp'], label=f"AUC = {result['AUC']:.2f}")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve {result['Model']} with sampling: {result['Sampling']}")
    plt.legend(loc=4)
    plt.show()

    print("\n" + "="*60 + "\n")

##Considerazioni sui risultati

> Model: Reg. lineare with sampling: No sampling

AUC e cross-validation AUC simili agli altri, ma la precision sulla classe "1" è molto bassa con recall pari a zero, quindi si può dire che il modello fatica a identificare i casi positivi

> Model: Reg. lineare with sampling: SMOTE

Rispetto al caso precedente, sembra che il modello catturi in maniera più efficace i casi positivi, ma è anche aumentato il numero di falsi positivi

> Model: Reg. lineare with sampling: UnderSampling

Nessun risultato particolarmente diverso da quanto ottenuto con SMOTE.

> Model: Reg. lineare (balanced) with sampling: No sampling

Miglioramento dei parametri rispetto alla regressione non bilanciata per quanto riguarda il recall

> Model: Reg. lineare (balanced) with sampling: SMOTE

Rispetto al precedente c'è solo un leggerissimo miglioramento del parametro AUC

> Model: Reg. lineare (balanced) with sampling: UnderSampling

Come il precedente, nessun miglioramento o peggioramento rilevante


---
Basandosi sul modello bilanciato con i pesi, possiamo concludere di aver ottenuto un AUC di 0.83, che indica una buona capacità predittiva, ache se non ottima.

Il modello è efficace nell'identificare i casi positivi, ma restituisce anche molti falsi positivi, quindi potrebbe essere stimato che alcui clienti vogliano stipulare una assicurazione per il veicolo, ma non accadrà.

L'accuratezza media complessiva del modello è del 64%.

---

## Valutazione threshold ottimale

Partendo dal modello migliore, quindi quello bilanciato con sampling SMOTE, si usa la funzione "precision_recall_curve", per trovare il valore di soglia ottimale per questo modello.

Con questo valore, si rieffettua la valutazione del modello con la funzione definita in precedenza "evaluate_model".

A parità di AUC, si ottiene una precision leggermente migliore nel predire il caso positivo, ma a scapito del recall.

Questo significa che clienti potenzialmente interessate potrebbero essere omessi dalle iniziative di marketing. Considerata la differenza minima e che una proposta di acquisto non accettata non comporta danni, non si ritiene utile variare il threshold.

In [ ]:
from sklearn.metrics import precision_recall_curve, f1_score
import numpy as np

model_final = LogisticRegression(random_state=RANDOM_SEED, class_weight='balanced')
x_train_balanced, y_train_balanced = SMOTE(random_state=RANDOM_SEED).fit_resample(x_train, y_train)
model_final.fit(x_train_balanced, y_train_balanced)

y_pred_proba = model_final.predict_proba(x_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Optimal Threshold: {optimal_threshold}")

name_model = f"Regr. lineare bilanciata (con sampling SMOTE) - THRESHOLD 0.65"
result_evaluation_model_optimal_threshold = evaluate_model(model_final, x_train_balanced, y_train_balanced, x_test, y_test, name_model, threshold=optimal_threshold)

print(name_model)
print(f"AUC: {result_evaluation_model_optimal_threshold['AUC']}")
print(f"Cross-Validation AUC: {result_evaluation_model_optimal_threshold['CrossValidationAUC']}")
print(f"Confusion Matrix:\n{result_evaluation_model_optimal_threshold['ConfusionMatrix']}")
print(f"Classification Report:\n{result_evaluation_model_optimal_threshold['ClassificationReport']}")

plt.figure()
plt.plot(result_evaluation_model_optimal_threshold['fp'], result_evaluation_model_optimal_threshold['tp'], label=f"AUC = {result_evaluation_model_optimal_threshold['AUC']:.2f}")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"ROC Curve")
plt.legend(loc=4)
plt.show()